In [1]:
import pandas as pd
import glob
import warnings
import arcpy
warnings.filterwarnings('ignore')
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
import os
import geopandas as gpd
import rasterio
import rasterio.mask
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import TwoSlopeNorm, LinearSegmentedColormap

In [2]:
gis_path = r'F:/SEACAR_WQ_2024/GIS_Data/'

In [3]:
dfweek_diff = pd.read_csv(gis_path + 'week_difference.csv', low_memory=False)
dfweek_diff_drop = dfweek_diff.dropna(subset=["ConValue"]).reset_index(drop=True)
dfweek_diff_group = dfweek_diff_drop.groupby(["WaterBody","ParameterName","Latitude_DD","Longitude_DD",
                                              "x","y","WbodyAcronym","ParaAcronym"]).agg({"ResultValue":"mean","ConValue":"mean"}).reset_index()
dfweek_diff_group["Difference"] = dfweek_diff_group["ConValue"] - dfweek_diff_group["ResultValue"]
wb_list   = list(dfweek_diff_group.WbodyAcronym.unique())
para_list = list(dfweek_diff_group.ParaAcronym.unique())

In [4]:
def kernel_density(workspace,weekormonth,df,search_radius):
    for wb in wb_list:
        for para in para_list:
            env.workspace = workspace
            name = str(wb)+"_"+str(para)
            weekormonth = str(weekormonth)
            search_radius = int(search_radius)
            df_use = df[(df["WbodyAcronym"]==str(wb))&(df["ParaAcronym"]==str(para))]
            df_use.to_csv(env.workspace+f"/{weekormonth}/{name}.csv")
            arcpy.management.XYTableToPoint(env.workspace+f"/{weekormonth}/{name}.csv", env.workspace+f"/{weekormonth}/{name}.shp",
                                "x", "y", "Difference",arcpy.SpatialReference(3086))    
            print(name,arcpy.management.GetCount(env.workspace+f"/{weekormonth}/{name}.shp"))
            outKDens = KernelDensity(in_features=env.workspace+f"/{weekormonth}/{name}.shp", population_field="Difference",search_radius=search_radius)
            output_tif = env.workspace + f"/{weekormonth}/{name}_kd.tif"
            outKDens.save(output_tif)

In [5]:
kernel_density(gis_path+"temp","week",dfweek_diff_group,5000)

BB_DO_mgl 45
BB_Sal_ppt 30
BB_Turb_ntu 30
BB_T_c 47
CH_DO_mgl 388
CH_Sal_ppt 442
CH_Turb_ntu 37
CH_T_c 467
EB_DO_mgl 92
EB_Sal_ppt 19
EB_Turb_ntu 60
EB_T_c 104
GTM_DO_mgl 72
GTM_Sal_ppt 69
GTM_Turb_ntu 6
GTM_T_c 69


In [6]:
dfmonth_diff = pd.read_csv(gis_path + 'month_difference.csv', low_memory=False)
dfmonth_diff_drop = dfmonth_diff.dropna(subset=["ConValue"]).reset_index(drop=True)
dfmonth_diff_group = dfmonth_diff_drop.groupby(["WaterBody","ParameterName","Latitude_DD","Longitude_DD",
                                                "x","y","WbodyAcronym","ParaAcronym"]).agg({"ResultValue":"mean","ConValue":"mean"}).reset_index()
dfmonth_diff_group["Difference"] = dfmonth_diff_group["ConValue"] - dfmonth_diff_group["ResultValue"]
wb_list   = list(dfmonth_diff_group.WbodyAcronym.unique())
para_list = list(dfmonth_diff_group.ParaAcronym.unique())

In [7]:
kernel_density(gis_path+"temp","month",dfmonth_diff_group,5000)

BB_DO_mgl 45
BB_Sal_ppt 30
BB_Turb_ntu 30
BB_T_c 47
CH_DO_mgl 303
CH_Sal_ppt 428
CH_Turb_ntu 29
CH_T_c 450
EB_DO_mgl 70
EB_Sal_ppt 13
EB_Turb_ntu 32
EB_T_c 72
GTM_DO_mgl 70
GTM_Sal_ppt 67
GTM_Turb_ntu 5
GTM_T_c 67
